In [1]:
import icechunk
import zarr

Lets create an in-memory icechunk store

In [2]:
repo = icechunk.Repository.create(
    storage=icechunk.StorageConfig.memory("")
)
repo

Ok! Lets create some data!

In [3]:
session = repo.writable_session("main")
store = session.store()
group = zarr.group(store=store, overwrite=True)
group

<Group <icechunk.store.IcechunkStore object at 0x116706b90>>

In [4]:
air_temp = group.create_array(
    "air_temp", shape=(1000, 1000), chunk_shape=(100, 100), dtype="i4"
)
air_temp

<Array <icechunk.store.IcechunkStore object at 0x116706b90>/air_temp shape=(1000, 1000) dtype=int32>

In [5]:
async for key in store.list():
    print(key)

zarr.json
air_temp/zarr.json


In [6]:
air_temp[:, :] = 42

In [7]:
air_temp[200, 6]

array(42, dtype=int32)

Now that we have set the values, lets commit

In [8]:
snapshot_id = session.commit("Initial commit")
snapshot_id

'QS2Z02V6HEQ3PZYB31BG'

Lets get another session

In [9]:
from typing import cast


session = repo.writable_session("main")
store = session.store()
group = zarr.open_group(store)
air_temp = cast(zarr.Array, group["air_temp"])

Okay now we can change the data

In [10]:
air_temp[:, :] = 54

In [11]:
air_temp[200, 6]

array(54, dtype=int32)

And we can commit again

In [12]:
new_snapshot_id = session.commit("Change air temp to 54")
new_snapshot_id

'KKWDTZQDHS3G86D797BG'

Cool, now lets checkout the original snapshot and see if the value is 42 again

In [13]:
store = repo.readonly_session(snapshot_id=new_snapshot_id).store()
group = zarr.open_group(store, mode='r')
air_temp = cast(zarr.Array, group["air_temp"])
air_temp[200, 6]

array(54, dtype=int32)